# **Sentiment Analysis**

The `IMDb reviews` dataset is the “hello world” of natural language processing: it consists of
50,000 movie reviews in English (25,000 for training, 25,000 for testing)You can load the IMDB dataset easily:

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

tf.random.set_seed(42)

# load train and test data
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

X_train[0][:10]
#


[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

The dataset is already preprocessed for you: X_train consists of a list of reviews, each of which
is represented as a NumPy array of integers, where each integer represents
a word. All punctuation was removed, and then words were converted to
lowercase, split by spaces, and finally indexed by frequency (so low
integers correspond to frequent words). The integers 0, 1, and 2 are
special: they represent the padding(relleno) token, the start-of-sequence (SSS)
token, and unknown words, respectively. If you want to visualize a review,
you can decode it like this:


In [2]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token


" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

En un proyecto real, deberá preprocesar el texto usted mismo. Puede hacerlo usando la
misma clase `Tokenizer` que usamos anteriormente, pero esta vez configurando
char_level=False (que es el valor predeterminado). 
 
Ademas psts implementar su modelo en un dispositivo móvil o en un navegador web, y no quiere
tener que escribir una función de preprocesamiento diferente cada vez, querrá manejar
el preprocesamiento usando solo operaciones de TensorFlow, para que pueda incluirse
en el modelo mismo. Veamos cómo. Primero, carguemos las reseñas originales de IMDb,
como texto (cadenas de bytes), usando `TensorFlow Datasets`

In [3]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
datasets.keys()


e:\codes\ia_tf212\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dict_keys(['train', 'test', 'unsupervised'])

In [4]:
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

print(train_size, test_size)


25000 25000


In [5]:
# mostrar las dos primeras reviews
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:100], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside.  ...
Label: 0 = Negative

Review: I have been known to fall asleep during films, but this is usually due to a combination of things in ...
Label: 0 = Negative



## Preprocessing data

It starts by truncating the reviews, keeping only the first 300 characters of
each: this will speed up training, and it won’t impact performance too
much because you can generally tell whether a review is positive or not in
the first sentence or two. Then it uses regular expressions to replace <\br
/> tags with spaces, and to replace any characters other than letters and
quotes with spaces. 
 
Finally, the preprocess() function splits the
reviews by the spaces, which returns a ragged tensor, and it converts this
ragged tensor to a dense tensor, padding all reviews with the padding
token "<pad>" so that they all have the same length

In [6]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

# ejemplo de como quedan las palabras dividivas 
preprocess(X_batch, y_batch)


(<tf.Tensor: shape=(2, 53), dtype=string, numpy=
 array([[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie',
         b"Don't", b'be', b'lured', b'in', b'by', b'Christopher',
         b'Walken', b'or', b'Michael', b'Ironside', b'Both', b'are',
         b'great', b'actors', b'but', b'this', b'must', b'simply', b'be',
         b'their', b'worst', b'role', b'in', b'history', b'Even',
         b'their', b'great', b'acting', b'could', b'not', b'redeem',
         b'this', b"movie's", b'ridiculous', b'storyline', b'This',
         b'movie', b'is', b'an', b'early', b'nineties', b'US',
         b'propaganda', b'pi', b'<pad>', b'<pad>', b'<pad>'],
        [b'I', b'have', b'been', b'known', b'to', b'fall', b'asleep',
         b'during', b'films', b'but', b'this', b'is', b'usually', b'due',
         b'to', b'a', b'combination', b'of', b'things', b'including',
         b'really', b'tired', b'being', b'warm', b'and', b'comfortable',
         b'on', b'the', b'sette', b'and', b'having', b'j

Next, we need to construct the vocabulary. This requires going through the
whole training set once, applying our preprocess() function, and using a
`Counter` to count the number of occurrences of each word

In [7]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))


print("the most common words: ",vocabulary.most_common()[:3])

the most common words:  [(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]


In [8]:
len(vocabulary)


53893

Quedarnos con las palabras mas frequentes

In [9]:
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]


In [10]:
# Ejemplo de tokenizacion 
word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}
for word in b"This movie was faaaaaantastic ".split():
    print(word_to_id.get(word) or vocab_size)


22
12
11
10000


Ahora necesitamos agregar un paso de preprocesamiento para reemplazar cada palabra
con su ID (es decir, su índice en el vocabulario). Al igual que hicimos en el `Capítulo 13`,
se creara  `lookup table` for this, using 1,000 out-of-vocabulary (oov)
buckets:

In [12]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [15]:
table.lookup(tf.constant([b"This movie was faaaaaantastic <pad>".split()]))

<tf.Tensor: shape=(1, 5), dtype=int64, numpy=array([[   22,    12,    11, 10053,     0]], dtype=int64)>

In [14]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch


train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

Notar que cada batch tendra el mismo tamaño de sequencia pq se uso en la funcion process `to_tensor(default_value=b"<pad>"`

In [19]:
for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)


tf.Tensor(
[[  22   11   28 ...    0    0    0]
 [   6   21   70 ...    0    0    0]
 [4099 6881    1 ...    0    0    0]
 ...
 [  22   12  118 ...  331 1047    0]
 [1757 4101  451 ...    0    0    0]
 [3365 4392    6 ...    0    0    0]], shape=(32, 60), dtype=int64)
tf.Tensor([0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0], shape=(32,), dtype=int64)


## Building and training model

La primera capa es una capa de incrustación, que convertirá las identificaciones de palabras
en incrustaciones (introducidas en el Capítulo 13). La matriz de incrustación debe tener una
fila por ID de palabra (vocab_size + num_oov_buckets) y una columna por dimensión de
incrustación (este ejemplo usa 128 dimensiones, pero este es un hiperparámetro que puede
ajustar). Mientras que las entradas del modelo serán tensores de forma 2D **[batch size, time steps]**, la salida de la capa
de incrustación será un tensor de forma 3D **[batch size, time steps,embedding size]**.

In [20]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True,  # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),

    keras.layers.GRU(128),  # devuelve solo la salida del último paso de tiempo.
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 50s 57ms/step - loss: 0.5309 - accuracy: 0.7264
Epoch 2/5
782/782 [==============================] - 44s 57ms/step - loss: 0.3410 - accuracy: 0.8583
Epoch 3/5
782/782 [==============================] - 44s 56ms/step - loss: 0.1802 - accuracy: 0.9367
Epoch 4/5
782/782 [==============================] - 44s 56ms/step - loss: 0.1383 - accuracy: 0.9516
Epoch 5/5
782/782 [==============================] - 44s 56ms/step - loss: 0.1115 - accuracy: 0.9590


## Manual masking:

El modelo deberá aprender que las fichas de relleno `<pad>` deben
ignorarse. ¡Pero eso ya lo sabemos! ¿Por qué no le decimos al modelo que ignore los
tokens de relleno, para que pueda concentrarse en los datos que realmente importan? En realidad, es trivial: simplemente agregue `mask_zero=True` al crear la capa Embedding(incrustación).
Esto significa que todas las capas descendentes ignorarán los tokens de relleno (cuyo ID es
0)
 
 The way this works is that the Embedding layer creates a mask tensor
equal to `K.not_equal(inputs, 0)` *(where K = keras.backend)*: it is a
Boolean tensor with the same shape as the inputs, and it is equal to False
anywhere the word IDs are 0, or True other wise. This mask tensor is then
automatically propagated by the model to all subsequent layers, as long as
the time dimension is preserved. 

Entonces, en este ejemplo, ambas capas GRU recibirán esta
máscara automáticamente, pero dado que la segunda capa GRU no devuelve secuencias
(solo devuelve la salida del último paso de tiempo), la máscara no se transmitirá a la
capa Densa. 

 Each layer may handle the mask
differently, but in general they simply ignore masked time steps (i.e., time
steps for which the mask is False). For example, when a recurrent layer
encounters a masked time step, it simply copies the output from the
previous time step.

In [21]:
K = keras.backend
embed_size = 128
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)

model = keras.models.Model(inputs=[inputs], outputs=[outputs])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# history = model.fit(train_set, epochs=5)



Después de entrenar durante algunas épocas, el modelo será bastante bueno para
juzgar si una review es positiva o no. Si usa TensorBoard()
callback,puede visualizar las incrustaciones en TensorBoard a medida que se aprenden: es
fascinante ver palabras como *"impresionante"* y *"asombroso"* agruparse
gradualmente en un lado del espacio de incrustación, mientras que palabras como *"
horrible"* y *"terrible"* grupo en el otro lado. Algunas palabras no son
tan positivo como cabría esperar (al menos con este modelo), como la palabra *"bueno"*,
presumiblemente porque muchas críticas negativas contienen la frase *"no bueno"*.

___
The `LSTM` and `GRU` layers have an optimized implementation for GPUs, based on
Nvidia’s cuDNN library. However, this implementation does not support `masking`. If
your model uses a mask, then these layers will fall back to the (much slower) default
implementation. Note that the optimized implementation also requires you to use the
default values for several hyperparameters: **activation, recurrent_activation,
recurrent_dropout, unroll, use_bias, and reset_after**.
____

  

## **Reusing Pretrained Embeddings**
 
Se puede reutilizar **Embeddings** de palabras entrenadas en algún otro corpus de texto grande (por ejemplo, artículos de
Wikipedia). Después de
todo, la palabra "increíble" generalmente tiene el mismo significado ya sea que la uses para
hablar de películas o de cualquier otra cosa. Además, tal vez las incrustaciones
serían útiles para el análisis de sentimientos incluso si se entrenaron en otra tarea: dado
que palabras como "impresionante" y "sorprendente" tienen un significado similar, es
probable que se agrupen en el espacio de incrustación incluso para otras tareas 


De forma predeterminada, TF Hub almacenará en caché los archivos descargados en el
directorio temporal del sistema local. Es posible que prefiera descargarlos en un directorio
más permanente para evitar tener que descargarlos nuevamente después de cada limpieza
del sistema. Para ello, establezca la variable de entorno `TFHUB_CACHE_DIR` en
el directorio de su elección (p. ej., os.environ["TFHUB_CACHE_DIR"] ="./my_tfhub_cache")

In [7]:
import os
import tensorflow as tf
tf.random.set_seed(42)

TFHUB_CACHE_DIR = os.path.join(os.curdir, "my_tfhub_cache")
os.environ["TFHUB_CACHE_DIR"] = TFHUB_CACHE_DIR

El proyecto `TensorFlow Hub` facilita la reutilización de componentes de modelos
previamente entrenados en sus propios modelos, denominados módulos. Simplemente navegue por el repositorio de TF Hub, encuentre el que
necesita y copie el ejemplo de código en su proyecto, y el módulo será descargado automáticamente con sus pesos entrenados. 

Por ejemplo, usemos el módulo de incrustación de oraciones `nnlm en dim50, versión 1`, en
nuestro modelo de análisis de sentimiento:

This particular module is a **sentence encoder**: it takes strings as input and
encodes each one as a single vector (in this case, a 50-dimensional vector).Internamente, analiza la cadena (dividiendo palabras en espacios) e incrusta cada palabra usando
una matriz de incrustación que fue previamente entrenada en un corpus enorme: el corpus 7B de
Google News (¡siete mil millones de palabras!). Luego calcula la media de todas las incrustaciones
de palabras y el resultado es la incrustación de oraciones

In [ ]:
import tensorflow_hub as hub

model = keras.Sequential([

    # By default, a hub.KerasLayer is not trainable,
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]),

    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])


In [ ]:
for dirpath, dirnames, filenames in os.walk(TFHUB_CACHE_DIR):
    for filename in filenames:
        print(os.path.join(dirpath, filename))


./my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe.descriptor.txt
./my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe/saved_model.pb
./my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe/variables/variables.data-00000-of-00001
./my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe/variables/variables.index
./my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe/assets/tokens.txt


Next, we can just load the IMDb reviews dataset—no need to preprocess it
(except for batching and prefetching)—and directly train the model:

In [ ]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].batch(batch_size).prefetch(1)
history = model.fit(train_set, epochs=5)


Epoch 1/5
782/782 [==============================] - 128s 164ms/step - loss: 0.5460 - accuracy: 0.7267
Epoch 2/5
782/782 [==============================] - 128s 164ms/step - loss: 0.5129 - accuracy: 0.7495
Epoch 3/5
782/782 [==============================] - 129s 165ms/step - loss: 0.5082 - accuracy: 0.7530
Epoch 4/5
782/782 [==============================] - 128s 164ms/step - loss: 0.5047 - accuracy: 0.7533
Epoch 5/5
782/782 [==============================] - 128s 164ms/step - loss: 0.5015 - accuracy: 0.7560
